In [ ]:
from heapq import merge

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy import feature as cfeature

In [ ]:
equipment_data = pd.read_csv('../data/equipment.csv')
meldung_data = pd.read_csv('../data/Locker_Meldungen_Aufträge.csv')
anzahl_meldungen = pd.read_csv('../data/Anzahl_Meldungen.csv', delimiter=';')
kosten_data = pd.read_csv('../data/Locker_Kosten_Erlöse.csv')

In [ ]:
meldung_data.info()

In [ ]:
meldung_data.head()

In [ ]:
equipment_data.info()

In [ ]:
def extract_id(old: pd.Series) -> pd.Series:
    print(old.str.split(' ').str[1])
    new = old.str.split(' ').str[1]
    return new.astype(int)

In [ ]:
filter_i = meldung_data["Equipment"].str.contains("LÖVM")
meldung_data = meldung_data[~filter_i]

In [ ]:
anzahl_meldungen.Equipment = extract_id(anzahl_meldungen.Equipment)

In [ ]:
meldung_data.Equipment = extract_id(meldung_data.Equipment)

In [ ]:
meldung_data.info()

In [ ]:
anzahl_meldungen["Equipment"].astype(int)
anzahl_meldungen.info()

In [ ]:
#anzahl_meldungen = meldung_data.groupby("Equipment")["Meldung"].count().reset_index()
anzahl_meldungen.rename(columns={"Anzahl Sätze": "Anzahl Meldungen"}, inplace=True)

In [ ]:
merged_df = pd.merge(equipment_data, anzahl_meldungen, how='left', on='Equipment')

In [ ]:
merged_df.rename(columns={"Equipment": "Equipment ID"}, inplace=True)

In [ ]:
merged_df.info()

In [ ]:
columns_to_keep = ["Equipment ID", "Anzahl Meldungen", "lat", "lon", "Zyklus Wartung"]
merged_df = merged_df[columns_to_keep]

In [ ]:
# Replace Zyklus Wartung with Numbers
def zyklus_in_int(zyklus: str) -> int:
    if zyklus == "6M":
        zyklus = 6 * 30
    elif zyklus == "1J":
        zyklus = 365
    elif zyklus == "2J":
        zyklus = 365 * 2
    elif zyklus == "4J":
        zyklus = 365 * 4
    return zyklus

In [ ]:
merged_df["Zyklus Wartung"] = merged_df["Zyklus Wartung"].apply(zyklus_in_int)

In [ ]:
print(np.round(merged_df.max(), 2), "\n", np.round(merged_df.min(), 2))


In [ ]:
merged_df.to_csv('../data/merged_data.csv', index=False)